# Belongs To High School Data

### Introduction
This database we are looking at in this lab has two tables, `sat_records` and `high_schools`. Each row of the `sat_records` table represents a school also found in the `high_schools` table. Both tables have a key in the column "dbn" that is a unique identifier for each school, and will be our way of connecting the data between the two tables. We do this by using a JOIN clause.

Let's begin by connecting to the database and reviewing the structure of the tables:

In [1]:
import sqlite3
conn = sqlite3.connect('nyc_schools.db')
cursor = conn.cursor()

In [2]:
cursor.execute('SELECT name from sqlite_master where type= "table"')
cursor.fetchall()

[('sat_records',), ('high_schools',)]

In [3]:
cursor.execute('PRAGMA table_info(high_schools)')
cursor.fetchall()

[(0, 'id', 'INTEGER', 0, None, 0),
 (1, 'dbn', 'TEXT', 0, None, 0),
 (2, 'school_name', 'TEXT', 0, None, 0),
 (3, 'boro', 'TEXT', 0, None, 0),
 (4, 'total_students', 'INTEGER', 0, None, 0),
 (5, 'graduation_rate', 'REAL', 0, None, 0),
 (6, 'attendance_rate', 'REAL', 0, None, 0),
 (7, 'college_career_rate', 'REAL', 0, None, 0)]

In [4]:
cursor.execute('PRAGMA table_info(sat_records)')
cursor.fetchall()

[(0, 'id', 'INTEGER', 0, None, 0),
 (1, 'dbn', 'TEXT', 0, None, 0),
 (2, 'name', 'TEXT', 0, None, 0),
 (3, 'num_test_takers', 'REAL', 0, None, 0),
 (4, 'reading_avg', 'REAL', 0, None, 0),
 (5, 'math_avg', 'REAL', 0, None, 0),
 (6, 'writing_score', 'REAL', 0, None, 0)]

Each of the following questions require information in both of the tables in the database. Use the JOIN clause to find the solution.

* In which boro is the school that has the highest writing score?

In [5]:
cursor.execute("""
SELECT schools.boro,
       MAX(sat.writing_score) 
  FROM sat_records AS sat
       JOIN
       high_schools AS schools ON sat.dbn = schools.dbn;

""")
cursor.fetchall()

[('M', 682.0)]

* In which boro is the school with the lowest math average?

In [6]:
cursor.execute("""
SELECT schools.boro,
       MIN(sat.math_avg) 
  FROM sat_records AS sat
       JOIN
       high_schools AS schools ON sat.dbn = schools.dbn;

""")
cursor.fetchall()

[('X', 312.0)]

* What is the highest math_avg for schools with more than 1000 students?

In [7]:
cursor.execute("""
SELECT MAX(sat.math_avg) 
  FROM sat_records AS sat
       JOIN
       high_schools AS schools ON sat.dbn = schools.dbn
 WHERE schools.total_students > 1000;

""")
cursor.fetchall()

[(735.0,)]

* What is the average number of test takers in each boro?

In [8]:
cursor.execute("""
SELECT schools.boro,
       AVG(sat.num_test_takers) 
  FROM sat_records AS sat
       JOIN
       high_schools AS schools ON sat.dbn = schools.dbn
 GROUP BY schools.boro;

""")
cursor.fetchall()

[('K', 126.33673469387755),
 ('M', 110.34177215189874),
 ('Q', 199.51666666666668),
 ('R', 300.5),
 ('X', 80.3875)]

* What is the attendance rate of schools with math_avg greater than 500? Order your results by the attendance rate (descending)

In [ ]:
cursor.execute("""
SELECT schools.attendance_rate
  FROM sat_records AS sat
       JOIN
       high_schools AS schools ON sat.dbn = schools.dbn
 WHERE sat.math_avg > 500
 ORDER BY schools.attendance_rate DESC;
""")
# cursor.fetchall()

* What is the graduation rate of schools with a math_avg less than 500? Order your results by the graduation rate (ascending)

In [ ]:
cursor.execute("""
SELECT schools.graduation_rate
  FROM sat_records AS sat
       JOIN
       high_schools AS schools ON sat.dbn = schools.dbn
 WHERE sat.math_avg < 500
 ORDER BY schools.graduation_rate;
 """)
cursor.fetchall()

* For schools with a math avg greater than 500, what is the average graduation rate?

In [11]:
cursor.execute("""
SELECT AVG(schools.graduation_rate) 
  FROM sat_records AS sat
       JOIN
       high_schools AS schools ON sat.dbn = schools.dbn
 WHERE sat.math_avg > 500;

""")
cursor.fetchall()

[(0.9769999999999999,)]

* What is the total number of test takers in each boro?

In [12]:
cursor.execute("""
SELECT schools.boro,
       SUM(sat.num_test_takers) 
  FROM sat_records AS sat
       JOIN
       high_schools AS schools ON sat.dbn = schools.dbn
 GROUP BY schools.boro;
""")
cursor.fetchall()

[('K', 12381.0), ('M', 8717.0), ('Q', 11971.0), ('R', 3005.0), ('X', 6431.0)]

* What is the average combined reading and math scores for each boro?

In [13]:
cursor.execute("""
SELECT schools.boro,
       AVG(sat.math_avg + sat.reading_avg) 
  FROM sat_records AS sat
       JOIN
       high_schools AS schools ON sat.dbn = schools.dbn
 GROUP BY schools.boro;

""")
cursor.fetchall()

[('K', 795.2857142857143),
 ('M', 869.5822784810126),
 ('Q', 874.5666666666667),
 ('R', 930.0),
 ('X', 778.2375)]

* Find the top five schools that have the largest differences between num_test_takers and total_students

In [14]:
cursor.execute("""
SELECT schools.school_name,
       (schools.total_students - sat.num_test_takers) AS difference
  FROM sat_records AS sat
       JOIN
       high_schools AS schools ON sat.dbn = schools.dbn
 ORDER BY difference DESC
 LIMIT 5;
""")
cursor.fetchall()

[('Brooklyn Technical High School', 4561.0),
 ('Fort Hamilton High School', 3888.0),
 ('Francis Lewis High School', 3623.0),
 ('Midwood High School', 3234.0),
 ('James Madison High School', 3139.0)]

* What is the difference between total students and number of test takers for each boro? Order your answer in ascending order 

In [15]:
cursor.execute("""
SELECT schools.boro,
       SUM(schools.total_students - sat.num_test_takers) AS difference
  FROM sat_records AS sat
       JOIN
       high_schools AS schools ON sat.dbn = schools.dbn
 GROUP BY schools.boro
 ORDER BY difference;

""")
cursor.fetchall()

[('R', 15627.0),
 ('X', 35972.0),
 ('M', 41118.0),
 ('Q', 58712.0),
 ('K', 61192.0)]